## In this notebook:


* 11 features to predict clinvar
* 

### Imports

In [2]:
#!pip install numpy 
#!pip install pandas
#!pip install torch
#!pip install sklearn

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
import torch.optim as optim

### Manipulating data
* Import csv we want
* Map our features to usuable integers
* Map our labels to a one hot encoding

In [3]:
#################################################### 
df = pd.read_pickle("./pickled/chr2-4_featureset2.pkl")
df = df.replace({'-': 0})

# Split our data into three separate sets for training purposes
train, test = train_test_split(df, shuffle=False)

print(test.head())
print(train.shape, test.shape)

                      aapos  APPRIS TSL codonpos  Ancestral_allele  \
chr pos      ref alt                                                 
3   97791800 T   G      170       5   5        2                 2   
    98580709 G   A      447       5   1        1                 2   
    98581474 T   C      404       5   1        1                 2   
    98585559 G   A      352       5   1        1                 2   
    98585622 G   A      331       5   1        1                 2   

                     HUVEC_fitCons_score      ExAC_cnv.score         GDI  \
chr pos      ref alt                                                       
3   97791800 T   G              0.655142  -0.151628993663861    24.13311   
    98580709 G   A              0.714379   0.240023872494554  3622.54592   
    98581474 T   C              0.714379   0.240023872494554  3622.54592   
    98585559 G   A              0.714379   0.240023872494554  3622.54592   
    98585622 G   A              0.714379   0.24002387

### DataLoader
* Implement the Dataset class supplied by PyTorch
  * Used Map style
  * API: https://pytorch.org/docs/stable/data.html#map-style-datasets
* Instanciate DataLoader using Dataset instance

In [4]:
class DataSetdbNSFP(Dataset):
    def __init__(self, data):
        self.data = torch.FloatTensor(data.values.astype('float'))
        
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
            target = self.data[index][-1]
            data_val = self.data[index] [:-1]
            return data_val,target
        
train_dataset = DataSetdbNSFP(train)
valid_dataset = DataSetdbNSFP(train)  
test_dataset = DataSetdbNSFP(test)

train_loader = DataLoader(train_dataset, batch_size=59, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=59, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=59, shuffle=True)


### Model
* This model is a simple implementation
* Based on https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [8]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # In: nb of features, out: nb of examples
        self.fc = nn.Linear(11, 1000)
        self.fc2 = nn.Linear(1000, 1000)
        self.fc3 = nn.Linear(1000, 1000)
        self.fc4 = nn.Linear(1000, 1000)
         # In: nb of examples, out: nb of predictions
        self.fc5 = nn.Linear(1000, 2)

    def forward(self, x):
        x = x.view((-1, 11))
        h = F.relu(self.fc(x))
        h = self.fc2(h)
        h = F.relu(self.fc2(h))
        h = self.fc3(h)
        h = F.relu(self.fc3(h))
        h = self.fc4(h)
        h = F.relu(self.fc4(h))
        h = self.fc5(h)
        # Softmax to get the actual labels
        return F.softmax(h, dim=1)  ## Note: can potentially use this to get confidence of category predicted 
    
model = Model()
# if cuda:
#     model.cuda()

# Optimizer based on Adam algorithm, uses a slightly lower rate
optimizer = optim.Adam(model.parameters(), lr=1e-4)

### Training
* Run a fixed number of epochs
* After each epoch, run the validation set to adjust learning
* Print results
* Based on example: https://github.com/ieee8023/NeuralNetwork-Examples/blob/master/pytorch/pytorch-mnist.ipynb

In [9]:
EPOCHS = 20

model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        print(data.shape)
        print(target.shape)
        
#         if cuda:
#             data, target = data.cuda(), target.cuda()
        
        optimizer.zero_grad()
        y_pred = model(data) 
        
        # Calculate and appy loss
        loss = F.cross_entropy(y_pred, target.long())
        loss.backward()
        optimizer.step()

        print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch+1,
            EPOCHS,
            batch_idx * len(data), 
            len(train_loader.dataset),
            100. * batch_idx / len(train_loader), 
            loss.cpu().data.item()))
    
    # NOTE: The Variable class is a wrapper class around a tensor with the added functionalities of back propagation
    # For x, take all cols of datafram except the last one
    evaluate_x = Variable(valid_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
    print(evaluate_x)
    # For y, take last col
    evaluate_y = Variable(valid_loader.dataset.data[:, -1:])
#     if cuda:
#         evaluate_x, evaluate_y = evaluate_x.cuda(), evaluate_y.cuda()
    model.eval()
    output = model(evaluate_x)
    pred = output.data.max(1)[1]
    
    y_labels = torch.flatten(evaluate_y.data)
    print('Predictions:', pred)
    print('Actual values:', y_labels)
    d = pred.eq(y_labels).cpu()
    accuracy = d.sum().item()*1./d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Validation Accuracy: {:.2f}%'.format(
        epoch+1,
        EPOCHS,
        len(train_loader.dataset), 
        len(train_loader.dataset),
        100. * batch_idx / len(train_loader), 
        loss.cpu().data.item(),
        accuracy*100))

torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [0/10300 (0%)]	Loss: 0.620239
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [59/10300 (1%)]	Loss: 0.604527
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [118/10300 (1%)]	Loss: 0.506806
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [177/10300 (2%)]	Loss: 0.567706
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [236/10300 (2%)]	Loss: 0.492592
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [295/10300 (3%)]	Loss: 0.591999
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [354/10300 (3%)]	Loss: 0.584468
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [413/10300 (4%)]	Loss: 0.605887
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [472/10300 (5%)]	Loss: 0.449560
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [531/10300 (5%)]	Loss: 0.567822
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [590/10300 (6%)]	Loss: 0.519487
torch.Size([59, 11])
tor

 Train Epoch: 1/20 [5428/10300 (53%)]	Loss: 0.636312
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [5487/10300 (53%)]	Loss: 0.601397
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [5546/10300 (54%)]	Loss: 0.635296
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [5605/10300 (54%)]	Loss: 0.703092
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [5664/10300 (55%)]	Loss: 0.618363
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [5723/10300 (55%)]	Loss: 0.533814
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [5782/10300 (56%)]	Loss: 0.736940
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [5841/10300 (57%)]	Loss: 0.551584
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [5900/10300 (57%)]	Loss: 0.601613
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [5959/10300 (58%)]	Loss: 0.635307
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 1/20 [6018/10300 (58%)]	Loss: 0.686143
torch.Size([59, 11])
torch.Size([59])

 Train Epoch: 2/20 [118/10300 (1%)]	Loss: 0.618347
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [177/10300 (2%)]	Loss: 0.550550
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [236/10300 (2%)]	Loss: 0.652245
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [295/10300 (3%)]	Loss: 0.618346
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [354/10300 (3%)]	Loss: 0.567499
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [413/10300 (4%)]	Loss: 0.669194
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [472/10300 (5%)]	Loss: 0.533600
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [531/10300 (5%)]	Loss: 0.601397
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [590/10300 (6%)]	Loss: 0.601397
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [649/10300 (6%)]	Loss: 0.567499
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [708/10300 (7%)]	Loss: 0.448971
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [7

 Train Epoch: 2/20 [5487/10300 (53%)]	Loss: 0.584448
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [5546/10300 (54%)]	Loss: 0.567499
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [5605/10300 (54%)]	Loss: 0.533600
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [5664/10300 (55%)]	Loss: 0.652245
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [5723/10300 (55%)]	Loss: 0.550550
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [5782/10300 (56%)]	Loss: 0.686143
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [5841/10300 (57%)]	Loss: 0.652243
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [5900/10300 (57%)]	Loss: 0.686143
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [5959/10300 (58%)]	Loss: 0.601397
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [6018/10300 (58%)]	Loss: 0.567499
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 2/20 [6077/10300 (59%)]	Loss: 0.550550
torch.Size([59, 11])
torch.Size([59])

 Train Epoch: 3/20 [59/10300 (1%)]	Loss: 0.499702
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 3/20 [118/10300 (1%)]	Loss: 0.567499
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 3/20 [177/10300 (2%)]	Loss: 0.567499
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 3/20 [236/10300 (2%)]	Loss: 0.584448
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 3/20 [295/10300 (3%)]	Loss: 0.567499
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 3/20 [354/10300 (3%)]	Loss: 0.533601
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 3/20 [413/10300 (4%)]	Loss: 0.482753
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 3/20 [472/10300 (5%)]	Loss: 0.533600
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 3/20 [531/10300 (5%)]	Loss: 0.635295
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 3/20 [590/10300 (6%)]	Loss: 0.635296
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 3/20 [649/10300 (6%)]	Loss: 0.584448
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 3/20 [70

...

Predictions: tensor([1, 1, 1,  ..., 1, 1, 1])
Actual values: tensor([0., 0., 0.,  ..., 1., 1., 1.])
 Train Epoch: 18/20 [10300/10300 (99%)]	Loss: 0.577968	 Validation Accuracy: 72.21%
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [0/10300 (0%)]	Loss: 0.550550
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [59/10300 (1%)]	Loss: 0.567499
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [118/10300 (1%)]	Loss: 0.431906
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [177/10300 (2%)]	Loss: 0.533600
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [236/10300 (2%)]	Loss: 0.652245
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [295/10300 (3%)]	Loss: 0.635296
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [354/10300 (3%)]	Loss: 0.635296
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [413/10300 (4%)]	Loss: 0.618346
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [472/10300 (5%)]	Loss: 0.567499
torch.Siz

 Train Epoch: 19/20 [5192/10300 (50%)]	Loss: 0.618346
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [5251/10300 (51%)]	Loss: 0.550550
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [5310/10300 (51%)]	Loss: 0.584448
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [5369/10300 (52%)]	Loss: 0.686143
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [5428/10300 (53%)]	Loss: 0.516651
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [5487/10300 (53%)]	Loss: 0.567499
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [5546/10300 (54%)]	Loss: 0.686143
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [5605/10300 (54%)]	Loss: 0.550550
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [5664/10300 (55%)]	Loss: 0.652245
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [5723/10300 (55%)]	Loss: 0.465804
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 19/20 [5782/10300 (56%)]	Loss: 0.618346
torch.Size([59, 11])
torch

Predictions: tensor([1, 1, 1,  ..., 1, 1, 1])
Actual values: tensor([0., 0., 0.,  ..., 1., 1., 1.])
 Train Epoch: 19/20 [10300/10300 (99%)]	Loss: 0.548556	 Validation Accuracy: 72.21%
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [0/10300 (0%)]	Loss: 0.601397
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [59/10300 (1%)]	Loss: 0.601397
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [118/10300 (1%)]	Loss: 0.516651
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [177/10300 (2%)]	Loss: 0.635295
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [236/10300 (2%)]	Loss: 0.516651
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [295/10300 (3%)]	Loss: 0.686143
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [354/10300 (3%)]	Loss: 0.601397
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [413/10300 (4%)]	Loss: 0.669194
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [472/10300 (5%)]	Loss: 0.584448
torch.Siz

 Train Epoch: 20/20 [5192/10300 (50%)]	Loss: 0.686143
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [5251/10300 (51%)]	Loss: 0.601397
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [5310/10300 (51%)]	Loss: 0.584448
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [5369/10300 (52%)]	Loss: 0.635296
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [5428/10300 (53%)]	Loss: 0.686143
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [5487/10300 (53%)]	Loss: 0.601397
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [5546/10300 (54%)]	Loss: 0.652245
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [5605/10300 (54%)]	Loss: 0.482753
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [5664/10300 (55%)]	Loss: 0.635296
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [5723/10300 (55%)]	Loss: 0.567499
torch.Size([59, 11])
torch.Size([59])
 Train Epoch: 20/20 [5782/10300 (56%)]	Loss: 0.601397
torch.Size([59, 11])
torch

Predictions: tensor([1, 1, 1,  ..., 1, 1, 1])
Actual values: tensor([0., 0., 0.,  ..., 1., 1., 1.])
 Train Epoch: 20/20 [10300/10300 (99%)]	Loss: 0.577968	 Validation Accuracy: 72.21%


### Testing
* Take testing dataset, run it against current model
* Evaluate testing accuracy

In [13]:
# Follow same steps as when we test with validation set, except with the test set
evaluate_x = Variable(test_loader.dataset.data[:, 0:-1].type_as(torch.FloatTensor()))
evaluate_y = Variable(test_loader.dataset.data[:, -1:])

model.eval()
output = model(evaluate_x)
pred = output.data.max(1)[1]


y_labels = torch.flatten(evaluate_y.data)
d = pred.eq(y_labels).cpu()
accuracy = d.sum().item()*1./d.size()[0]  ## Accuracy = (TP+TN)/(TP+TN+FP+FN)

print('\n Testing Accuracy: {:.2f}%'.format(

    accuracy*100))

print(len(pred))
print(len(y_labels))


 Testing Accuracy: 68.46%
3434
3434


### Metrics
* Use sklearn builtins to calculate different metrics

In [10]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
print("Confusion matrix: \n", confusion_matrix(y_labels, pred))

# Accuracy
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_labels, pred))

# Recall
from sklearn.metrics import recall_score
recall = recall_score(y_labels, pred, average=None)
print("Recall TP/(TP+FN):", recall)

# Precision
from sklearn.metrics import precision_score
precision = precision_score(y_labels, pred, average=None)
print("Precision TP/(TP+FP):",precision)


# F1 score
F1 = 2 * (precision * recall) / (precision + recall)
print("F1:", F1)

Confusion matrix: 
 [[   0 2862]
 [   0 7438]]
Accuracy: 0.722135922330097
Recall TP/(TP+FN): [0. 1.]
Precision TP/(TP+FP): [0.         0.72213592]
F1: [       nan 0.83865148]


/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
